In [ ]:
from dotenv import load_dotenv
from langchain_community.chat_models import ChatTongyi
from langchain_core.messages import HumanMessage

import overview.config as cfg
from overview.agent import main_agent, search_subagent
from overview.Analysis.PaperMeta import PaperMetaAnalyzer
from overview.tools.VectorStore import VectorStore
import overview.tools.memory as memory

load_dotenv()


True

In [3]:
vec_store = VectorStore(
    md_dir="./examples/Example/md",
    store_dir="./examples/Example/vectStore",
)

paper_info = PaperMetaAnalyzer(
    file_paths=["examples/Example/pdf/1706.03762v7.pdf"]
).metadata


sync_store completed
📄 正在分析: 1706.03762v7.pdf
DONE : examples/Example/pdf/1706.03762v7.pdf



In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
sub_model = ChatTongyi(model="qwen-max")
#model = ChatGoogleGenerativeAI(model="gemini-2.5-pro")
agent = main_agent(
    sub_model, subagents=[search_subagent(sub_model)], tools=[vec_store.retriever_tool]
)


In [5]:
from IPython.display import display, Markdown, HTML, clear_output
import time

def print_header(role, content, color):
    """打印带颜色的标题（适配黑色背景）"""
    # User 用亮绿色 (#00ff00), AI 用亮青色 (#00ffff)
    html_code = f"""
    <div style="border-left: 5px solid {color}; padding-left: 10px; margin-top: 10px;">
        <span style="color: {color}; font-weight: bold; font-size: 1.1em;">{role}</span>
        <br>
        <span style="color: #cccccc;">{content}</span>
    </div>
    """
    display(HTML(html_code))


def print_ai_markdown(content):
    """使用 Markdown 渲染 AI 的回复（支持公式和代码高亮）"""
    display(Markdown(content))
    display(HTML("<hr style='border-color: #333;'>"))  # 灰色分割线


def parse_ai_response(raw_content):
    if isinstance(raw_content, str):
        return raw_content
    elif isinstance(raw_content, list):
        # 拼接列表中所有的 'text' 字段
        text_parts = []
        for item in raw_content:
            if isinstance(item, dict) and "text" in item:
                text_parts.append(item["text"])
            elif isinstance(item, str):
                text_parts.append(item)
        return "".join(text_parts)
    else:
        return str(raw_content)


# --- 主逻辑 ---

print("🟢 交互开始 (输入 'exit' 结束)")

# 首次提问
question = input("请输入你的问题: ")

if question.strip().lower() not in ["exit", "quit", ""]:
    # 打印用户问题
    print_header("User", question, "#00ff00")  # 亮绿色
    history = cfg.BaseState(
        messages=[HumanMessage(content=question + f" PAPER_META_INFO: {paper_info}")]
    )
    # history = memory.resume("history.json")
    while True:
        try:
            # 显示 "思考中..." 提示
            loading_html = HTML(
                '<span style="color: #e6db74; font-style: italic;">🤖 AI 正在思考中...</span>'
            )
            display(loading_html)

            # 1. 调用 AI
            res = agent.invoke(history)
            raw_content = res["messages"][-1].content
            history["messages"] = res["messages"]
            # 2. 【修复点】解析内容，确保它是字符串
            ai_content = parse_ai_response(raw_content)

            # 清除 "思考中" 的提示（这里我们简单地忽略，因为直接打印结果会把它顶上去）

            # 3. 打印 AI 回复
            print_header("AI Assistant", "", "#00ffff")  # 亮青色头
            print_ai_markdown(ai_content)  # Markdown 正文

            # 4. 获取下一次输入
            question = input("请输入你的问题 (exit 退出): ")

            if question.strip().lower() in ["exit", "quit"]:
                print("👋 已退出交互")
                break

            # 打印新一轮的用户问题
            print_header("User", question, "#00ff00")

            history["messages"] += [HumanMessage(content=question)]

        except KeyboardInterrupt:
            print("\n🛑 用户强制中断")
            break
        except Exception as e:
            print(f"❌ 发生错误: {e}")
            if "raw_content" in locals():
                print(f"调试信息: 返回的数据类型是 {type(raw_content)}")
            break
else:
    print("👋 已退出")


🟢 交互开始 (输入 'exit' 结束)


这篇文章的主要内容如下：

- **标题**: GLU Variants Improve Transformer
- **发表年份**: 2020
- **引用次数**: 20
- **领导机构**: Google (United States)
- **OpenAlex页面链接**: [GLU Variants Improve Transformer](https://openalex.org/W3006439205)
- **PDF链接**: [arXiv PDF](https://arxiv.org/pdf/2002.05202)
- **作者**: Noam Shazeer
- **概念关键词**:
  - Sigmoid function
  - Transformer
  - 非线性系统
  - 数学
  - 序列（生物学）
  - 计算机科学
  - 算法
  - 人工智能
  - 工程
  - 电压
  - 物理
  - 电气工程
  - 遗传学
  - 量子力学
  - 生物学
  - 人工神经网络

- **过去三年的引用趋势**:
  - 2022: 2次
  - 2021: 16次
  - 2020: 2次

- **摘要**:
  Gated Linear Units (GLU) 包括两个线性投影的逐元素乘积，其中一个先通过一个Sigmoid函数。可以使用不同的非线性（甚至线性）函数代替Sigmoid来实现GLU的变体。我们在Transformer序列到序列模型的前馈子层中测试了这些变体，并发现其中一些变体在质量上优于通常使用的ReLU或GELU激活函数。

这篇文章主要探讨了GLU的不同变体在Transformer模型中的应用，并且实验表明某些变体相比常用的ReLU或GELU激活函数能够带来性能上的提升。

👋 已退出交互


In [6]:
import overview.tools.memory as memory

memory.save("history.json", history)
history

✅ Save history to history.json


BaseState(messages=[HumanMessage(content="这篇文章的主要内容是什么 PAPER_META_INFO: [PaperMetadata(title='GLU Variants Improve Transformer', publication_year=2020, cited_by_count=20, institution_lead='Google (United States)', openalex_url='https://openalex.org/W3006439205', pdf_url='https://arxiv.org/pdf/2002.05202', authors=['Noam Shazeer'], concepts=['Sigmoid function', 'Transformer', 'Nonlinear system', 'Mathematics', 'Sequence (biology)', 'Computer science', 'Algorithm', 'Artificial intelligence', 'Engineering', 'Voltage', 'Physics', 'Electrical engineering', 'Genetics', 'Quantum mechanics', 'Biology', 'Artificial neural network'], citation_trend_last_3y={2022: 2, 2021: 16, 2020: 2}, abstract_full='Gated Linear Units (arXiv:1612.08083) consist of the component-wise product of two linear projections, one of which is first passed through a sigmoid function. Variations on GLU are possible, using different nonlinear (or even linear) functions in place of sigmoid. We test these variants in the feed

In [7]:
import overview.tools.memory as memory

state = memory.resume("history.json")

state["messages"] += [HumanMessage(content="我们刚刚聊了什么？")]

agent.invoke(state)

{'messages': [HumanMessage(content="这篇文章的主要内容是什么 PAPER_META_INFO: [PaperMetadata(title='GLU Variants Improve Transformer', publication_year=2020, cited_by_count=20, institution_lead='Google (United States)', openalex_url='https://openalex.org/W3006439205', pdf_url='https://arxiv.org/pdf/2002.05202', authors=['Noam Shazeer'], concepts=['Sigmoid function', 'Transformer', 'Nonlinear system', 'Mathematics', 'Sequence (biology)', 'Computer science', 'Algorithm', 'Artificial intelligence', 'Engineering', 'Voltage', 'Physics', 'Electrical engineering', 'Genetics', 'Quantum mechanics', 'Biology', 'Artificial neural network'], citation_trend_last_3y={2022: 2, 2021: 16, 2020: 2}, abstract_full='Gated Linear Units (arXiv:1612.08083) consist of the component-wise product of two linear projections, one of which is first passed through a sigmoid function. Variations on GLU are possible, using different nonlinear (or even linear) functions in place of sigmoid. We test these variants in the feed-forwa